In [1]:
# Install PaddlePaddle (GPU version)
!python -m pip install paddlepaddle-gpu==2.6.1 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html

# Install PaddleOCR
!pip install "paddleocr>=2.7.0" # Rekomendasi versi terbaru

# Install dependensi lain jika diperlukan (PaddleOCR biasanya sudah menyertakan)
!pip install Pillow pandas

Looking in links: https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.9/758.9 MB 1.0 MB/s eta 0:00:0000:0100:10
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: opt-einsum
    Found existing installation: opt_einsum 3.4.0
    Uninstalling opt_einsum-3.4.0:
      Successfully uninstalled opt_einsum-3.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!git clone https://github.com/PaddlePaddle/PaddleOCR.git
%cd PaddleOCR
!pip install -r requirements.txt
%cd ..

Cloning into 'PaddleOCR'...
remote: Enumerating objects: 232107, done.
remote: Counting objects: 100% (1241/1241), done.
remote: Compressing objects: 100% (274/274), done.
remote: Total 232107 (delta 970), reused 1192 (delta 943), pack-reused 230866 (from 1)
Receiving objects: 100% (232107/232107), 1.22 GiB | 23.69 MiB/s, done.
Resolving deltas: 100% (182907/182907), done.
/kaggle/working/PaddleOCR
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.8/297.8 kB 5.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 51.5 MB/s eta 0:00:00a 0:00:01
/kaggle/working


In [3]:
import os
import pandas as pd
import random

# Path ke font Anda (sesuaikan dengan nama dataset Kaggle Anda)
FONT_DIR_KAGGLE = "/kaggle/input/fonts-receipt" # GANTI 'my-custom-fonts' jika nama dataset Anda berbeda

# Path ke dataset OpenFoodFacts
OPENFOODFACTS_CSV = "/kaggle/input/openfoodfacts-export-csv/openfoodfacts_export_csv.csv"

# Direktori output untuk dataset sintetis
SYNTHETIC_DATA_DIR = "/kaggle/working/synthetic_receipt_dataset/"
IMAGES_OUTPUT_DIR = os.path.join(SYNTHETIC_DATA_DIR, "images")
LABEL_FILE_PATH = os.path.join(SYNTHETIC_DATA_DIR, "train_label.txt")
DICT_FILE_PATH = os.path.join(SYNTHETIC_DATA_DIR, "custom_dict.txt")

os.makedirs(IMAGES_OUTPUT_DIR, exist_ok=True)

# Path untuk data training di dalam struktur PaddleOCR
PADDLEOCR_TRAIN_DATA_DIR = "/kaggle/working/PaddleOCR/train_data/my_receipts/"
os.makedirs(os.path.join(PADDLEOCR_TRAIN_DATA_DIR, "images"), exist_ok=True)

In [4]:
# Fungsi untuk menghasilkan harga acak
def generate_random_price():
    # Hasilkan angka antara 1000 dan 500000
    price_val = random.randint(1000, 500000)
    # Format dengan titik sebagai pemisah ribuan
    return f"Rp {price_val:,}".replace(',', '.')

corpus_lines = []

# Baca nama produk dari OpenFoodFacts
try:
    df_food = pd.read_csv(OPENFOODFACTS_CSV, usecols=['product_name_en'], low_memory=False)
    # Ambil nama produk yang tidak kosong dan merupakan string
    product_names = df_food['product_name_en'].dropna().astype(str).unique().tolist()
    # Filter nama produk yang terlalu pendek atau terlalu panjang, dan bersihkan
    product_names = [name.strip() for name in product_names if 3 < len(name.strip()) < 50]
    product_names = [name for name in product_names if '\t' not in name and '\n' not in name] # Hapus tab/newline
    corpus_lines.extend(product_names)
    print(f"Berhasil memuat {len(product_names)} nama produk unik dari OpenFoodFacts.")
except Exception as e:
    print(f"Error saat membaca OpenFoodFacts CSV: {e}")
    print("Menggunakan beberapa contoh nama produk saja.")
    corpus_lines.extend([
        "Susu UHT Cokelat", "Mie Instan Goreng Spesial", "Kopi Bubuk Robusta",
        "Teh Celup Melati", "Biskuit Kelapa Renyah"
    ]) # Fallback jika CSV gagal dimuat

# Tambahkan data harga ke corpus
num_prices_to_generate = len(corpus_lines) # Buat jumlah harga yang sama dengan produk, atau sesuaikan
for _ in range(num_prices_to_generate):
    corpus_lines.append(generate_random_price())

random.shuffle(corpus_lines) # Acak urutan baris corpus

if not corpus_lines:
    raise ValueError("Corpus kosong! Periksa pembacaan CSV atau data fallback.")

print(f"Total {len(corpus_lines)} baris dalam corpus (produk dan harga).")
print("Contoh beberapa baris corpus:")
for i in range(min(10, len(corpus_lines))):
    print(corpus_lines[i])

# Simpan corpus ke file (opsional, tapi bisa berguna)
with open("/kaggle/working/invoice_corpus.txt", 'w', encoding='utf-8') as f:
    for line in corpus_lines:
        f.write(line + '\n')

Error saat membaca OpenFoodFacts CSV: Usecols do not match columns, columns expected but not found: ['product_name_en']
Menggunakan beberapa contoh nama produk saja.
Total 10 baris dalam corpus (produk dan harga).
Contoh beberapa baris corpus:
Teh Celup Melati
Susu UHT Cokelat
Rp 276.836
Rp 23.034
Rp 339.103
Rp 341.577
Mie Instan Goreng Spesial
Biskuit Kelapa Renyah
Kopi Bubuk Robusta
Rp 344.228


In [5]:
from PIL import Image, ImageDraw, ImageFont
import glob

# --- Konfigurasi Image Generation ---
IMAGES_PER_FONT = 500
IMAGE_WIDTH = 600  # Lebar gambar (bisa disesuaikan)
# Tinggi gambar akan disesuaikan dengan font dan padding
PADDING = 15 # Padding atas-bawah dan kiri-kanan

# Warna
BACKGROUND_COLORS = [(245, 245, 245), (250, 250, 250), (255, 255, 255)]
TEXT_COLORS = [(0,0,0), (20,20,20), (40,40,40)]

# --- Fungsi Helper ---
def get_font_files_from_kaggle(font_dir_kaggle):
    font_files = []
    if not os.path.exists(font_dir_kaggle):
        print(f"PERINGATAN: Direktori font KAGGLE '{font_dir_kaggle}' tidak ditemukan!")
        print("Pastikan Anda telah menambahkan dataset font Anda ke notebook.")
        print("Menggunakan font default sistem jika tersedia (mungkin tidak ideal).")
        # Mencoba mencari font sistem sederhana sebagai fallback (kualitas mungkin rendah)
        try:
            # Di Kaggle, beberapa font standar mungkin ada.
            # Ini hanya upaya terbaik, idealnya gunakan font kustom Anda.
            font_files.append(ImageFont.truetype("DejaVuSans.ttf", 25).path)
        except IOError:
            print("Tidak bisa menemukan font sistem DejaVuSans.ttf.")
            # Jika Anda tahu path font lain di Kaggle, coba itu.
            # Atau, unggah font sebagai dataset.
    else:
        for ext in ('*.ttf', '*.TTF', '*.otf', '*.OTF'):
            font_files.extend(glob.glob(os.path.join(font_dir_kaggle, ext)))

    if not font_files:
        print("Tidak ada file font yang ditemukan. Proses pembuatan gambar mungkin gagal atau menggunakan font default yang sangat dasar.")
        # Sebagai fallback paling akhir, PIL mungkin punya font built-in yang sangat terbatas
        # Namun, ini tidak akan menggunakan 7 font kustom Anda.
        # Untuk tujuan demo, kita coba biarkan PIL handle jika font tidak ketemu, tapi ini tidak ideal.
    return font_files

all_generated_labels_data = []
generated_image_count = 0

font_paths_kaggle = get_font_files_from_kaggle(FONT_DIR_KAGGLE)
if not font_paths_kaggle:
    print("KRITIKAL: Tidak ada font yang bisa digunakan. Silakan perbaiki path FONT_DIR_KAGGLE atau unggah font Anda.")
else:
    print(f"Menggunakan {len(font_paths_kaggle)} font dari Kaggle: {font_paths_kaggle}")

    for font_path in font_paths_kaggle:
        font_name_prefix = os.path.splitext(os.path.basename(font_path))[0]
        print(f"\nMemproses font: {font_name_prefix}...")
        current_font_images = 0
        try:
            # Coba muat font untuk memastikan valid sebelum loop utama
            _ = ImageFont.truetype(font_path, 28)
        except Exception as e:
            print(f"  Tidak bisa memuat font {font_path}: {e}. Melewati font ini.")
            continue

        for i in range(IMAGES_PER_FONT):
            if not corpus_lines: # Jika corpus habis
                print("Corpus habis, berhenti generate.")
                break

            text_to_render = random.choice(corpus_lines) # Ambil satu baris acak
            font_size = random.randint(26, 34) # Ukuran font acak

            try:
                font = ImageFont.truetype(font_path, font_size)
            except Exception as e:
                print(f"  Error memuat font {font_path} dengan size {font_size}: {e}. Melewati iterasi ini.")
                continue

            # Dapatkan ukuran teks untuk menentukan tinggi gambar
            try:
                # Pillow 9.2.0+
                text_bbox = font.getbbox(text_to_render)
                text_width = text_bbox[2] - text_bbox[0]
                text_height = text_bbox[3] - text_bbox[1]
            except AttributeError:
                # Versi Pillow lama
                text_width, text_height = font.getsize(text_to_render)


            img_height = text_height + (2 * PADDING)
            # Lebar gambar bisa disesuaikan agar pas dengan teks, atau tetap IMAGE_WIDTH
            # current_img_width = text_width + (2 * PADDING) # Opsi 1: Lebar pas dengan teks
            current_img_width = max(IMAGE_WIDTH, text_width + (2 * PADDING)) # Opsi 2: Minimal IMAGE_WIDTH

            image = Image.new('RGB', (int(current_img_width), int(img_height)), random.choice(BACKGROUND_COLORS))
            draw = ImageDraw.Draw(image)
            text_color = random.choice(TEXT_COLORS)

            # Posisi teks di tengah gambar
            text_x = (current_img_width - text_width) / 2
            text_y = PADDING - text_bbox[1] if 'text_bbox' in locals() else PADDING # Penyesuaian untuk y jika getbbox ada

            draw.text((text_x, text_y), text_to_render, font=font, fill=text_color)

            image_filename = f"{font_name_prefix}_img_{i:04d}.png"
            image_save_path = os.path.join(IMAGES_OUTPUT_DIR, image_filename)
            image.save(image_save_path)

            # Path relatif untuk file label PaddleOCR
            relative_image_path = os.path.join("images", image_filename) # Path relatif thd label file
            all_generated_labels_data.append(f"{relative_image_path}\t{text_to_render}")
            generated_image_count += 1
            current_font_images +=1

            if (i + 1) % 100 == 0:
                print(f"  Font {font_name_prefix}: {i + 1}/{IMAGES_PER_FONT} gambar dihasilkan.")

        if current_font_images < IMAGES_PER_FONT and not corpus_lines:
             print(f"  Font {font_name_prefix}: Corpus habis, hanya {current_font_images} gambar dihasilkan untuk font ini.")


# Tulis ke file label
with open(LABEL_FILE_PATH, 'w', encoding='utf-8') as f:
    for label_entry in all_generated_labels_data:
        f.write(label_entry + "\n")

print(f"\nTotal {generated_image_count} gambar berhasil dibuat di: {IMAGES_OUTPUT_DIR}")
print(f"File label dibuat di: {LABEL_FILE_PATH}")

# Membuat kamus karakter (dictionary)
print("\nMembuat file kamus karakter (custom_dict.txt)...")
all_chars = set()
for line_entry in all_generated_labels_data:
    try:
        _, text_label = line_entry.split('\t', 1)
        for char in text_label:
            all_chars.add(char)
    except ValueError:
        print(f"Peringatan: Baris label tidak valid: {line_entry}")


# Pastikan karakter ' ' (spasi) ada jika memang digunakan
# Urutkan karakter untuk konsistensi, ini penting untuk PaddleOCR
# Model PaddleOCR biasanya mengharapkan kamus diurutkan, dengan karakter blank (tidak terlihat) ditangani secara internal atau sebagai kelas terakhir.
# Beberapa implementasi menambahkan token khusus seperti <unk> atau <pad> di awal/akhir dict.
# Untuk CTCLoss, blank token ditangani secara internal. Dict hanya berisi karakter yang terlihat.

sorted_chars = sorted(list(all_chars))
with open(DICT_FILE_PATH, 'w', encoding='utf-8') as f:
    for char in sorted_chars:
        f.write(char + '\n')
print(f"File kamus 'custom_dict.txt' dibuat di {DICT_FILE_PATH} dengan {len(sorted_chars)} karakter.")
if not sorted_chars:
    print("PERINGATAN: Kamus karakter kosong! Ini akan menyebabkan masalah saat training.")

Menggunakan 7 font dari Kaggle: ['/kaggle/input/fonts-receipt/Monaco.ttf', '/kaggle/input/fonts-receipt/MerchantCopyDoublesize-jE7R.ttf', '/kaggle/input/fonts-receipt/DejaVuSans.ttf', '/kaggle/input/fonts-receipt/MerchantCopy-GOXq.ttf', '/kaggle/input/fonts-receipt/MerchantCopyWide-z8m0.ttf', '/kaggle/input/fonts-receipt/Epson Pixeled.ttf', '/kaggle/input/fonts-receipt/DOTMATRI.TTF']

Memproses font: Monaco...
  Font Monaco: 100/500 gambar dihasilkan.
  Font Monaco: 200/500 gambar dihasilkan.
  Font Monaco: 300/500 gambar dihasilkan.
  Font Monaco: 400/500 gambar dihasilkan.
  Font Monaco: 500/500 gambar dihasilkan.

Memproses font: MerchantCopyDoublesize-jE7R...
  Font MerchantCopyDoublesize-jE7R: 100/500 gambar dihasilkan.
  Font MerchantCopyDoublesize-jE7R: 200/500 gambar dihasilkan.
  Font MerchantCopyDoublesize-jE7R: 300/500 gambar dihasilkan.
  Font MerchantCopyDoublesize-jE7R: 400/500 gambar dihasilkan.
  Font MerchantCopyDoublesize-jE7R: 500/500 gambar dihasilkan.

Memproses fo

In [6]:
# Salin gambar, label, dan dict ke direktori training PaddleOCR
!cp -r {IMAGES_OUTPUT_DIR}/* {os.path.join(PADDLEOCR_TRAIN_DATA_DIR, "images")}/
!cp {LABEL_FILE_PATH} {PADDLEOCR_TRAIN_DATA_DIR}/
!cp {DICT_FILE_PATH} {PADDLEOCR_TRAIN_DATA_DIR}/

print(f"Data training disalin ke {PADDLEOCR_TRAIN_DATA_DIR}")
print("Isi direktori training PaddleOCR:")
!ls -l {PADDLEOCR_TRAIN_DATA_DIR}
print("\nContoh isi folder images di training PaddleOCR:")
!ls -l {os.path.join(PADDLEOCR_TRAIN_DATA_DIR, "images")} | head -n 5

Data training disalin ke /kaggle/working/PaddleOCR/train_data/my_receipts/
Isi direktori training PaddleOCR:
total 372
-rw-r--r-- 1 root root     78 Jun  2 08:08 custom_dict.txt
drwxr-xr-x 2 root root 196608 Jun  2 08:08 images
-rw-r--r-- 1 root root 178612 Jun  2 08:08 train_label.txt

Contoh isi folder images di training PaddleOCR:
total 19668
-rw-r--r-- 1 root root 4619 Jun  2 08:08 DejaVuSans_img_0000.png
-rw-r--r-- 1 root root 2999 Jun  2 08:08 DejaVuSans_img_0001.png
-rw-r--r-- 1 root root 4563 Jun  2 08:08 DejaVuSans_img_0002.png
-rw-r--r-- 1 root root 4834 Jun  2 08:08 DejaVuSans_img_0003.png
ls: write error: Broken pipe


In [7]:
%cd /kaggle/working/PaddleOCR/
!mkdir -p ./pretrain_models/
# Download English PP-OCRv4 recognition model
!wget -P ./pretrain_models/ https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_train.tar
!tar -xf ./pretrain_models/en_PP-OCRv4_rec_train.tar -C ./pretrain_models/
# Model akan diekstrak ke ./pretrain_models/en_PP-OCRv4_rec_train/
# Path ke file bobot biasanya best_accuracy.pdparams atau latest.pdparams
PRETRAINED_MODEL_PATH = "./pretrain_models/en_PP-OCRv4_rec_train/best_accuracy.pdparams" # Path lengkap ke file bobot
if not os.path.exists(PRETRAINED_MODEL_PATH):
    # Coba path alternatif jika struktur berbeda atau nama file beda
    PRETRAINED_MODEL_PATH = "./pretrain_models/en_PP-OCRv4_rec_train/latest.pdparams"
    if not os.path.exists(PRETRAINED_MODEL_PATH):
         print(f"Peringatan: File pretrained model di {PRETRAINED_MODEL_PATH} atau best_accuracy.pdparams tidak ditemukan setelah ekstraksi.")
else:
    print(f"Pretrained model siap di: {PRETRAINED_MODEL_PATH}")

%cd /kaggle/working/ # Kembali ke direktori kerja utama

/kaggle/working/PaddleOCR
--2025-06-02 08:08:11--  https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_train.tar
Resolving paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 103.235.47.176, 2402:2b40:7000:628:0:ff:b0e8:88da
Connecting to paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)|103.235.47.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190740480 (182M) [application/x-tar]
Saving to: ‘./pretrain_models/en_PP-OCRv4_rec_train.tar’

en_PP-OCRv4_rec_tra 100%[===================>] 181.90M  63.6MB/s    in 2.9s    

2025-06-02 08:08:14 (63.6 MB/s) - ‘./pretrain_models/en_PP-OCRv4_rec_train.tar’ saved [190740480/190740480]

Pretrained model siap di: ./pretrain_models/en_PP-OCRv4_rec_train/best_accuracy.pdparams
[Errno 2] No such file or directory: '/kaggle/working/ # Kembali ke direktori kerja utama'
/kaggle/working/PaddleOCR


In [8]:
# Hitung num_classes dari custom_dict.txt
with open(os.path.join(PADDLEOCR_TRAIN_DATA_DIR, "custom_dict.txt"), 'r', encoding='utf-8') as f:
    num_unique_chars = len(f.readlines())
# num_classes untuk CTCLoss = jumlah karakter unik + 1 (untuk blank token)
NUM_CLASSES = num_unique_chars + 1
print(f"Jumlah karakter unik dari dict: {num_unique_chars}, sehingga num_classes: {NUM_CLASSES}")

# Pilih file konfigurasi dasar, misal untuk English recognition PP-OCRv4
# Path di dalam repo PaddleOCR yang sudah di-clone
base_config_path = "/kaggle/working/PaddleOCR/configs/rec/PP-OCRv4/en_PP-OCRv4_mobile_rec.yml"
custom_config_path = "/kaggle/working/PaddleOCR/configs/rec/PP-OCRv4/my_receipt_config_kaggle.yml"

# Salin file konfigurasi dasar
!cp {base_config_path} {custom_config_path}

# Modifikasi file konfigurasi menggunakan Python atau sed
# Di Kaggle, memodifikasi YAML dengan Python lebih portabel
import yaml

with open(custom_config_path, 'r') as f:
    config = yaml.safe_load(f)

# --- Modifikasi Global ---
config['Global']['epoch_num'] = 50 # Kurangi epoch untuk Kaggle, atau sesuaikan
config['Global']['save_model_dir'] = "./output/rec_my_receipt_kaggle/"
config['Global']['pretrained_model'] = PRETRAINED_MODEL_PATH # Path lengkap ke .pdparams
config['Global']['character_dict_path'] = "train_data/my_receipts/custom_dict.txt" # Path relatif dari root PaddleOCR
config['Global']['num_classes'] = NUM_CLASSES
config['Global']['eval_batch_step'] = [0, 500] # Evaluasi setiap 500 batch, dan di awal
config['Global']['save_epoch_step'] = 5 # Simpan model setiap 5 epoch

# --- Modifikasi Optimizer ---
config['Optimizer']['lr']['learning_rate'] = 0.000001 # Learning rate lebih kecil untuk fine-tuning

# --- Modifikasi Train Dataset ---
config['Train']['dataset']['data_dir'] = "./train_data/my_receipts/" # Path relatif
config['Train']['dataset']['label_file_list'] = ["./train_data/my_receipts/train_label.txt"] # Path relatif
# Sesuaikan batch size dengan memori GPU Kaggle (P100 ~16GB, T4 ~15GB)
# Jika PP-OCRv4, image_shape [3, 48, 320] mungkin butuh batch size lebih kecil
config['Train']['loader']['batch_size_per_card'] = 128 # Coba, jika OOM, kurangi (misal 64, 96)
config['Train']['loader']['num_workers'] = 2 # Kaggle biasanya punya 2-4 core CPU efektif untuk loader

# --- Modifikasi Eval Dataset ---
# Untuk kesederhanaan, kita gunakan data training juga untuk evaluasi
# Idealnya, Anda punya set validasi terpisah
config['Eval']['dataset']['data_dir'] = "./train_data/my_receipts/"
config['Eval']['dataset']['label_file_list'] = ["./train_data/my_receipts/train_label.txt"]
config['Eval']['loader']['batch_size_per_card'] = 64
config['Eval']['loader']['num_workers'] = 2

# Jika ada parameter image_shape di config dasar, pastikan sesuai.
# Misal, untuk PP-OCRv4: RecResizeImg: image_shape: [3, 48, 320]
# Jika produk Anda sangat panjang, Anda mungkin perlu menambah lebar (dimensi ketiga).
# Untuk nama produk dan harga, [3, 48, 320] atau [3, 32, N] (jika model CRNN lama) mungkin cukup.
# PP-OCRv4 menggunakan tinggi 48.

# Tulis kembali konfigurasi yang sudah dimodifikasi
with open(custom_config_path, 'w') as f:
    yaml.dump(config, f, sort_keys=False)

print(f"File konfigurasi kustom disimpan di: {custom_config_path}")
print("--- Isi Konfigurasi Kustom (Global & Train) ---")
print("Global:", config['Global'])
print("Train Dataset:", config['Train']['dataset'])
print("Train Loader:", config['Train']['loader'])

Jumlah karakter unik dari dict: 39, sehingga num_classes: 40
File konfigurasi kustom disimpan di: /kaggle/working/PaddleOCR/configs/rec/PP-OCRv4/my_receipt_config_kaggle.yml
--- Isi Konfigurasi Kustom (Global & Train) ---
Global: {'model_name': 'en_PP-OCRv4_mobile_rec', 'debug': False, 'use_gpu': True, 'epoch_num': 50, 'log_smooth_window': 20, 'print_batch_step': 10, 'save_model_dir': './output/rec_my_receipt_kaggle/', 'save_epoch_step': 5, 'eval_batch_step': [0, 500], 'cal_metric_during_train': True, 'pretrained_model': './pretrain_models/en_PP-OCRv4_rec_train/best_accuracy.pdparams', 'checkpoints': None, 'save_inference_dir': None, 'use_visualdl': False, 'infer_img': 'doc/imgs_words/ch/word_1.jpg', 'character_dict_path': 'train_data/my_receipts/custom_dict.txt', 'max_text_length': 25, 'infer_mode': False, 'use_space_char': True, 'distributed': True, 'save_res_path': './output/rec/predicts_ppocrv3.txt', 'num_classes': 40}
Train Dataset: {'name': 'MultiScaleDataSet', 'ds_width': False,

In [9]:
%cd /kaggle/working/PaddleOCR/

# Perintah training (sesuai artikel dan praktik umum)
# -c path ke file konfigurasi Anda
# -o Global.load_static_weights=False penting saat fine-tuning dari model pre-trained
# Anda bisa menambahkan -o Global.use_gpu=true meskipun sudah ada di YAML untuk memastikan
!python tools/train.py \
    -c configs/rec/PP-OCRv4/my_receipt_config_kaggle.yml \
    -o Global.load_static_weights=False

/kaggle/working/PaddleOCR
[2025/06/02 08:08:22] ppocr WARNING: Skipping import of the encryption module.
[2025/06/02 08:08:22] ppocr INFO: Architecture : 
[2025/06/02 08:08:22] ppocr INFO:     Backbone : 
[2025/06/02 08:08:22] ppocr INFO:         name : PPLCNetV3
[2025/06/02 08:08:22] ppocr INFO:         scale : 0.95
[2025/06/02 08:08:22] ppocr INFO:     Head : 
[2025/06/02 08:08:22] ppocr INFO:         head_list : 
[2025/06/02 08:08:22] ppocr INFO:             CTCHead : 
[2025/06/02 08:08:22] ppocr INFO:                 Head : 
[2025/06/02 08:08:22] ppocr INFO:                     fc_decay : 1e-05
[2025/06/02 08:08:22] ppocr INFO:                 Neck : 
[2025/06/02 08:08:22] ppocr INFO:                     depth : 2
[2025/06/02 08:08:22] ppocr INFO:                     dims : 120
[2025/06/02 08:08:22] ppocr INFO:                     hidden_dims : 120
[2025/06/02 08:08:22] ppocr INFO:                     kernel_size : [1, 3]
[2025/06/02 08:08:22] ppocr INFO:                     name :

In [ ]:
# Ganti 'best_accuracy' dengan 'latest' jika itu yang terbaik/tersedia
# Path ke model terbaik hasil training
TRAINED_MODEL_BEST_PATH = "/kaggle/working/PaddleOCR/output/rec_my_receipt_kaggle/best_accuracy.pdparams" # atau latest.pdparams

# Cek apakah file ada
if not os.path.exists(TRAINED_MODEL_BEST_PATH):
    # Cari file .pdparams lain di direktori output jika 'best_accuracy' tidak ada
    output_model_dir = "/kaggle/working/PaddleOCR/output/rec_my_receipt_kaggle/"
    found_models = glob.glob(os.path.join(output_model_dir, "*.pdparams"))
    # Prioritaskan yang mengandung 'iter_epoch' atau 'latest' jika 'best_accuracy' tidak ada
    if any("latest" in m for m in found_models):
        TRAINED_MODEL_BEST_PATH = [m for m in found_models if "latest" in m][0]
    elif found_models: # Ambil saja salah satu jika ada
         TRAINED_MODEL_BEST_PATH = found_models[0] # Urutkan berdasarkan iterasi jika bisa
    else:
        print(f"Peringatan: Tidak ada file model .pdparams yang ditemukan di {output_model_dir}")
        TRAINED_MODEL_BEST_PATH = None # Set ke None jika tidak ada

if TRAINED_MODEL_BEST_PATH:
    print(f"Menggunakan model terlatih: {TRAINED_MODEL_BEST_PATH}")
    INFERENCE_MODEL_SAVE_DIR = "/kaggle/working/inference/my_receipt_rec_model_kaggle/"
    os.makedirs(INFERENCE_MODEL_SAVE_DIR, exist_ok=True)

    !python tools/export_model.py \
        -c configs/rec/PP-OCRv4/my_receipt_config_kaggle.yml \
        -o Global.pretrained_model={TRAINED_MODEL_BEST_PATH} \
        Global.save_inference_dir={INFERENCE_MODEL_SAVE_DIR}

    print(f"Model untuk inference disimpan di: {INFERENCE_MODEL_SAVE_DIR}")
    !ls -l {INFERENCE_MODEL_SAVE_DIR}
else:
    print("Tidak bisa mengekspor model karena tidak ada file model terlatih yang ditemukan.")

Menggunakan model terlatih: /kaggle/working/PaddleOCR/output/rec_my_receipt_kaggle/best_accuracy.pdparams
[2025/06/02 08:42:17] ppocr WARNING: Skipping import of the encryption module.
